# Constraint Optimization

## Objective and Prerequisites

If you are looking to improve your modeling skills, then try this tricky constraint optimization problem. We’ll show you how to model this problem as a linear programming problem using the Gurobi Python API and solve it using the Gurobi Optimizer.

This model is example 18 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 273 and 328-330.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 


---
## Problem Description

In an integer programming model, the following constraint occurs:

$$
9x_{1} + 13x_{2} -14x_{3} + 17x_{4} + 13x_{5} - 19x_{6} + 23x_{7} + 21x_{8} \leq 37
$$

All the decision variables in the constraint are binary, and the objective is to find another constraint involving the same binary variables that is logically equivalent to the original constraint, but that has the smallest possible absolute value of the right-hand side (RHS).

It is well established in the theory of  integer programming that the ability to efficiently solve an  integer programming problem depends on how "close" the linear programming relaxation is to the convex hull of all the integer solutions of the 
integer programming problem. A linear programming relaxation is "tight" when this formulation is "close" to the convex hull of all the integer solutions. We say that an equivalent constraint with respect to an original constraint is "tighter" whenever the equivalent constraint has the same integer solutions as the original one, but has removed some fractional feasible solutions of the original one. These ideas are illustrated in the following figure.

![OptimizedConstraint](OptimizedConstraint.PNG)

The original constraint in the figure is determined by the line AB and the "tighter" equivalent constraint is defined by the line CD. Notice that the equivalent constraint has the same set of integer solutions and has removed fractional feasible solutions of the original constraint.


---
## Model Formulation

A procedure for simplifying a single 0–1 constraint has been described by Bradley et al. (1974). We adopt their procedure of using a linear programming model. It is convenient to consider the constraint in a standard form with positive coefficients in descending order of magnitude. This can be achieved by the transformation:

$$
y_{1} = x_{7}, y_{2} = x_{8}, y_{3} = 1 - x_{6}, y_{4} = x_{4},
$$

$$
y_{5} = 1 - x_{3}, y_{6} = x_{5}, y_{7} = x_{2}, y_{8} = x_{1}.
$$

The resulting equivalent constraint is

$$
23y_{1} + 21y_{2} + 19y_{3} + 17y_{4} + 14y_{5} + 13y_{6} + 13y_{7} + 9y_{8} \leq 70.
$$

All inequalities of the form $\sum_{i=1}^{8} b_{i}*y_{i} \leq b_{0} $ equivalent to the constraint to be optimized are characterized by a set of inequalities of a linear programming model defined in terms of the decision variables $b_{i}$. Observe that the variables $b_{i}$ are the coefficients of the transformed constraint.

In order to capture the total logical import of the original constraint, we search for subsets of the indices known as *roofs* and *ceilings*. Ceilings are ‘maximal’ subsets of the indices of the variables for which the sum of the corresponding coefficients does not exceed the right-hand side coefficient. Such a subset is maximal in the sense that no subset properly containing it, or to the left in the implied lexicographical ordering, can also be a ceiling. For example, the subset {1, 2, 4, 8} is a ceiling, $23 +21 + 17 + 9 \leq 70$, but any subset properly containing it (e.g. {1, 2, 4, 7, 8}) or to the ‘left’ of it (e.g. {1, 2, 4, 7}) is not a ceiling. Roofs are ‘minimal’ subsets of the indices for which the sum of the corresponding coefficients exceeds the right-hand side coefficient. Such a subset is ‘minimal’ in the same sense as a subset is ‘maximal’. For example {2, 3, 4, 5} is a roof, 
$21 +19+ 17 + 14 > 70 $ ,
but any subset properly contained in it (e.g. {3, 4, 5}) or to the ‘right’ of it (e.g.
{2, 3, 4, 6}) is not a roof. See the details about how these constraints are derived in Bradley et al. (1974).

Therefore, all inequalities of the form $\sum_{i=1}^{8} b_{i}*y_{i} \leq b_{0} $ equivalent to the constraint to be optimized are characterized by the following constraints.

**Ceiling constraints**: <br />

$$
\{1,2,3 \}: b_{1} + b_{2} + b_{3} \leq b_{0}
$$

$$
\{1,2,4,8 \}: b_{1} + b_{2} + b_{4} + b_{8}  \leq b_{0}
$$

$$
\{1,2,6,7 \}: b_{1} + b_{2} + b_{6} + b_{7}  \leq b_{0}
$$

$$
\{1,3,5,6 \}: b_{1} + b_{3} + b_{5} + b_{6}  \leq b_{0}
$$

$$
\{2,3,4,6 \}: b_{2} + b_{3} + b_{4} + b_{6}  \leq b_{0}
$$

$$
\{2,5,6,7,8 \}: b_{2} + b_{5} + b_{6} + b_{7} + b_{8} \leq b_{0}
$$

**Roof constraints**: <br />

$$
\{1,2,3,8 \}: b_{1} + b_{2} + b_{3} + b_{8} \geq b_{0} + 1
$$

$$
\{1,2,5,7 \}: b_{1} + b_{2} + b_{5} + b_{7}  \geq b_{0} + 1
$$

$$
\{1,3,4,7 \}: b_{1} + b_{3} + b_{4} + b_{7} \geq b_{0} + 1
$$

$$
\{1,5,6,7,8 \}: b_{1} + b_{5} + b_{6} + b_{7} + b_{8} \geq b_{0} + 1
$$

$$
\{2,3,4,5 \}: b_{2} + b_{3} + b_{4} + b_{5} \geq b_{0} + 1
$$

$$
\{3,4,6,7,8 \}: b_{3} + b_{4} + b_{6} + b_{7} + b_{8} \geq b_{0} + 1
$$


$$
\text{Decreasing value:} \quad b_{i} \geq b_{i+1} \quad \forall i=1..7
$$

$$
\text{Non-negativity:} \quad b_{i} \geq 0 \quad \forall i=0,1..7
$$

The objective is to minimize the RHS of the original constraint.
$$
\text{Objective function:} \quad \text{Min} \;\; (b_{0} - b_{3} - b_{5} )
$$

---
## Python Implementation

We import the Gurobi Python Module.

In [ ]:
%pip install gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

In [2]:
# list of indices of variables b[i]

indices = [*range(9)]

## Model Deployment
We create a model and the decision variables. These variables determine the coefficients  of the transformed equivalent constraint that minimized the value of the RHS.

In [3]:
model = gp.Model('ConstraintOptimization')

# Variables b[i]
b = model.addVars(indices, name="b")

Using license file c:\gurobi\gurobi.lic


### Ceiling constraints

In [4]:
# Constraint {1, 2, 3}:

C123 = model.addConstr( b[1] + b[2] + b[3] <= b[0] , name='C123')

# Constraint {1, 2, 4, 8}}:

C1248 = model.addConstr( b[1] + b[2] + b[4] + b[8] <= b[0] , name='C1248')

# Constraint {1, 2, 6, 7}:

C1267 = model.addConstr( b[1] + b[2] + b[6] + b[7] <= b[0] , name='C1267')

# Constraint {1, 3, 5, 6}:

C1356 = model.addConstr( b[1] + b[3] + b[5] + b[6] <= b[0] , name='C1356')

# Constraint {2, 3, 4, 6}:

C2346 = model.addConstr( b[2] + b[3] + b[4] + b[6] <= b[0] , name='C2346')

# Constraint {2, 5, 6, 7, 8}:

C25678 = model.addConstr( b[2] + b[5] + b[6] + b[7] + b[8] <= b[0] , name='C25678')

### Roof constraints

In [5]:
# Constraint {1, 2, 3, 8}:

R1238 = model.addConstr( b[1] + b[2] + b[3] + b[8] >= b[0] + 1 , name='R1238')

# Constraint {1, 2, 5, 7}:

R1257 = model.addConstr( b[1] + b[2] + b[5] + b[7] >= b[0] + 1 , name='R1257')

# Constraint {1, 3, 4, 7}:

R1347 = model.addConstr( b[1] + b[3] + b[4] + b[7] >= b[0] + 1 , name='R1347')

# Constraint {1, 5, 6, 7, 8}:

R15678 = model.addConstr( b[1] + b[5] + b[6] + b[7] + b[8] >= b[0] + 1 , name='R15678')

# Constraint {2, 3, 4, 5}:

R2345 = model.addConstr( b[2] + b[3] + b[4] + b[5] >= b[0] + 1 , name='R2345')

# Constraint {3, 4, 6, 7, 8}:

R34678 = model.addConstr( b[3] + b[4] + b[6] + b[7] + b[8] >= b[0] + 1 , name='R34678')

### Decreasing value coefficients
These constraints guarantee the ordering of the coefficients.

In [6]:
# Decreasing value of coefficients

DV1_2 = model.addConstr( b[1] >= b[2] , name='DV1_2')
DV2_3 = model.addConstr( b[2] >= b[3] , name='DV2_3')
DV3_4 = model.addConstr( b[3] >= b[4] , name='DV3_4')
DV4_5 = model.addConstr( b[4] >= b[5] , name='DV4_5')
DV5_6 = model.addConstr( b[5] >= b[6] , name='DV5_6')
DV6_7 = model.addConstr( b[6] >= b[7] , name='DV6_7')
DV7_8 = model.addConstr( b[7] >= b[8] , name='DV7_8')

The objective is to minimize the  value of the RHS side of the original constraint.

In [7]:
# Objective function

model.setObjective(b[0] -b[3] -b[5] )

In [8]:
# Verify model formulation

model.write('OptimizeConstraint.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 19 rows, 9 columns and 76 nonzeros
Model fingerprint: 0x88e2e477
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 19 rows, 9 columns, 76 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+30   7.000000e+30   2.000000e+00      0s
      16    2.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  2.500000000e+01


In [9]:
# Output report


# Optimized values of the coefficients resepect to the original variables x.

a = {}

a[0] = b[0].x - b[3].x - b[5].x
a[1] = b[8].x
a[2] = b[7].x
a[3] = -b[5].x
a[4] = b[4].x
a[5] = b[6].x
a[6] = -b[3].x
a[7] = b[1].x
a[8] = b[2].x

print("________________________________________")
print(f"The equivalent optimized constraint is:")


optimized_constraint = {}
for i in indices:
    if i==0:
        optimized_constraint[i]=f"{a[i]}"
    else:
        optimized_constraint[i]= f"{a[i]}x[{i}]"
        #print(optimized_constraint)
            
print(f"\n{optimized_constraint[1]} + {optimized_constraint[2]} + {optimized_constraint[3]} + {optimized_constraint[4]} + {optimized_constraint[5]} + {optimized_constraint[6]}  + {optimized_constraint[7]} + {optimized_constraint[8]} <= {optimized_constraint[0]}")

________________________________________
The equivalent optimized constraint is:

6.0x[1] + 9.0x[2] + -10.0x[3] + 12.0x[4] + 9.0x[5] + -13.0x[6]  + 16.0x[7] + 14.0x[8] <= 25.0


---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Bradley, G.H., Hammer, P.L. and Wolsey, L. (1974) Coefficient Reduction for Inequalities in 0–1 Variables. Mathematical Programming, 7, 263–282.

Copyright © 2020 Gurobi Optimization, LLC